In [1]:
import tensorflow as tf

In [2]:
#Create tf dataset from a list
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [3]:
#Iterate through tf dataset

for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


In [4]:
#Iterate through elements as numpy elements
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


In [5]:
#Iterate through first n elements in tf dataset
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


In [6]:
#Filter sales numbers that are < 0

tf_dataset = tf_dataset.filter(lambda x: x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)



21
22
31
32
34
31


In [7]:
#Convert sales numbers from USA dollars ($) to Indian Rupees (INR) Assuming 1->72 conversation rate

tf_dataset = tf_dataset.map(lambda x: x*72)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


In [8]:
#Shuffe
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
2232
1584
2304
2232
2448


In [9]:
#Batching
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[1584 1512]
[2304 2232]
[2448 2232]


In [10]:
#Perform all of the above operations in one shot
#Since we are using two lambda function use two variables one in each
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1512 2232]
[1584 2448]
[2304 2232]


In [ ]:
#Images

In [ ]:
"""
creates a TensorFlow dataset containing file paths of images stored in a directory structure.
 
The images/ directory contains subdirectories (categories or labels).
*/* means:
First *: Matches any folder inside images/ (e.g., class names like cats/, dogs/).
Second *: Matches any file inside those folders (e.g., cat1.jpg, dog2.png).
"""
images_ds = tf.data.Dataset.list_files('images/*/*', shuffle=False)

In [ ]:
image_count = len(images_ds)
image_count

In [ ]:
type(images_ds)

In [ ]:
"""
Takes only three images and
Prints file paths of images in the 'images/' directory.
"""
for file in images_ds.take(3):
    print(file.numpy())

In [ ]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

In [ ]:
class_names = ["cat","dog"]

In [ ]:
"""
Since we shuffled cats and dogs are randomly jumbled
we take 80 percent as train set
we skip 80 percent and take 20 percent as test set
"""
train_size = int(image_count*0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [ ]:
len(train_ds)

In [ ]:
len(test_ds)

In [ ]:
"""
In the file path seperated by/the second from last has the subfolder name of cats or dogs which can be used to train
"""
def get_label(file_path):
    import os
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [ ]:
get_label("images\\dog\\20 Reasons Why Cats Make the Best Pets....jpg")

In [ ]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    
    #The output is a tensor with shape (height, width, channels), where channels=3 (RGB).
    img = tf.image.decode_jpeg(img) # Decode the raw bytes into a tensor
    
    img = tf.image.resize(img, [128, 128])
    return img, label

In [ ]:
img, label = process_image("images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg")
img.numpy()[:2]  # Extracts the first two rows of the image tensor using .numpy()[:2]

In [ ]:
"""
This applies the process_image function to each element in the train_ds and test_ds datasets, 
transforming file paths into processed images and labels.

train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [ ]:
#Take one image in the set and print its label and image
for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

In [ ]:
#Scale the image
def scale(image, label):
    return image/255, label

In [ ]:
#.map(scale) applies scale(image, label) to every (image, label) pair in train_ds.
train_ds = train_ds.map(scale)  

In [ ]:
"""
Take 5 elements from dataset and print the first top left pixel and its label
"""
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())